In [1]:
%load_ext autoreload
%autoreload 2

import pathlib

import pandas as pd
pd.set_option('display.float_format', lambda x: '%.1f' % x)
import scipy.stats
import numpy as np

import plotly.express as px
import plotly.graph_objs as go

from data_handlers.bwm import parse_bwms
from data_handlers.pidstat import parse_pidstat

In [2]:
experiment_path = pathlib.Path(f"/home/sterz/maci-docker-compose/maci_data/binary_files/17")

In [3]:
name_replacement = {
}

routing_color_map = {
}

In [4]:
def mean_confidence_interval(data, confidence=0.95):
    if isinstance(data.iloc[0], str):
        return data.iloc[0]
    
    a = 1.0 * np.array(data)
    n = len(a)
    h = scipy.stats.sem(a) * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return h

def error_band_line(error_y_mode=None, **kwargs):
    """Extension of `plotly.express.line` to use error bands."""
    ERROR_MODES = {'bar','band','bars','bands',None}
    if error_y_mode not in ERROR_MODES:
        raise ValueError(f"'error_y_mode' must be one of {ERROR_MODES}, received {repr(error_y_mode)}.")
    if error_y_mode in {'bar','bars',None}:
        fig = px.line(**kwargs)
    elif error_y_mode in {'band','bands'}:
        if 'error_y' not in kwargs:
            raise ValueError(f"If you provide argument 'error_y_mode' you must also provide 'error_y'.")
        figure_with_error_bars = px.line(**kwargs)
        fig = px.line(**{arg: val for arg,val in kwargs.items() if arg != 'error_y'})
        for data in figure_with_error_bars.data:
            x = list(data['x'])
            y_upper = list(data['y'] + data['error_y']['array'])
            y_lower = list(data['y'] - data['error_y']['array'] if data['error_y']['arrayminus'] is None else data['y'] - data['error_y']['arrayminus'])
            color = f"rgba({tuple(int(data['line']['color'].lstrip('#')[i:i+2], 16) for i in (0, 2, 4))},.3)".replace('((','(').replace('),',',').replace(' ','')
            fig.add_trace(
                go.Scatter(
                    x = x+x[::-1],
                    y = y_upper+y_lower[::-1],
                    fill = 'toself',
                    fillcolor = color,
                    line = dict(
                        color = 'rgba(255,255,255,0)'
                    ),
                    hoverinfo = "skip",
                    showlegend = False,
                    legendgroup = data['legendgroup'],
                    xaxis = data['xaxis'],
                    yaxis = data['yaxis'],
                )
            )
        # Reorder data as said here: https://stackoverflow.com/a/66854398/8849755
        reordered_data = []
        for i in range(int(len(fig.data)/2)):
            reordered_data.append(fig.data[i+int(len(fig.data)/2)])
            reordered_data.append(fig.data[i])
        fig.data = tuple(reordered_data)
    return fig

In [5]:
network_df = parse_bwms(experiment_path)

Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/17/1916
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/17/1882
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/17/1441
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/17/1436
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/17/1531
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/17/1888
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/17/1546
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/17/1866
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/17/1811
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/17/1645
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/17/1632
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/

/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/bwm.py:102: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby(["Software", "CLA", "Bundles per Second", "# Node", "# Payloads", "Payload Size", "dt"]).sum().reset_index()


In [6]:
procmem_df = parse_pidstat(experiment_path)

Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/17/1916
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/17/1882
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/17/1441
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/17/1436
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/17/1531
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/17/1888
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/17/1546
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/17/1866
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/17/1811
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/17/1645
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/17/1632
Parsing configuarion /home/sterz/maci-docker-compose/maci_data/binary_files/

/home/sterz/maci-docker-compose/maci_data/eval/data_handlers/pidstat.py:118: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby(["Software", "CLA", "Bundles per Second", "# Node", "# Payloads", "Payload Size", "dt"]).sum().reset_index()


In [48]:
merged = procmem_df.merge(network_df, left_index=True, right_index=True).reset_index()

In [49]:
cleared = merged[(merged["Bundles per Second"] == 100) & (merged["# Node"] == 32) & (merged["# Payloads"] == 100) & (merged["Payload Size"] == 100000)].copy()
cleared["dt"] = pd.to_timedelta(cleared["dt"], unit="S")
cleared.set_index("dt", inplace=True)
cleared_mean = cleared.groupby(["CLA"]).resample("1S").mean()

cleared_mean = cleared_mean.reset_index()

/tmp/ipykernel_1280570/4166593827.py:4: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

/tmp/ipykernel_1280570/4166593827.py:4: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [50]:
fig = error_band_line(
    data_frame=cleared_mean,
    x=cleared_mean["dt"].dt.total_seconds(),
    y="Mbit/s",
    # error_y=cleared_std["Mbit/s"],
    # error_y_mode="band",
    color="CLA",
)
fig.show()

In [51]:
fig = error_band_line(
    data_frame=cleared_mean,
    x=cleared_mean["dt"].dt.total_seconds(), 
    y="%CPU",
    # error_y=cleared_std["Mbit/s"],
    # error_y_mode="band",
    color="CLA",
)
fig.show()